In [ ]:
# imports and setup 
import pandas as pd
import scipy as sc
import numpy as np

import statsmodels.formula.api as sm

#%matplotlib notebook
import matplotlib.pyplot as plt 
plt.style.use('ggplot')
%matplotlib inline  
plt.rcParams['figure.figsize'] = (10, 6) 

In [ ]:
#df_pe_original = pd.read_csv('DKA - Physical.csv')
df_pe_original = pd.read_csv('PHYSEXAM.csv')

df_pe_original.head()

pd.set_option('display.max_columns', None) # View max columns
print(df_pe_original.columns.to_list())

df_pe_original.head()

In [ ]:
#Checking - are these all study initiation events? - yes
df_pe_original['StudyEvent'].value_counts() 
df_pe_original['PUDID'].nunique() #1389 - ok, one row per patient
#len(df_pe_original) #1389

This is now a markdown cell

# This cell is duplicated via function in the cell below now
df_pe = df_pe_original[['PUDID', 'PETime', 'PEHEENT', 'PEHEENTDesc', 'PECardio', 'PECardioDesc', 
                        'PEResp', 'PERespDesc', 'PEGI', 'PEGIDesc', 'PEHepatic', 'PEHepaticDesc', 
                        'PEGU', 'PEGUDesc', 'PERenal', 'PERenalDesc', 'PENeuro', 'PENeuroDesc', 
                        'PEPsych', 'PEPsychDesc', 'PEEndo', 'PEEndoDesc', 'PEHema', 'PEHemaDesc', 
                        'PEMusculo', 'PEMusculoDesc', 'PEDerm', 'PEDermDesc', 'PEAllergies', 
                        'PEAllergiesDesc', 'PEImmune', 'PEImmuneDesc', 'PEAlcohol', 'PEAlcoholDesc', 
                        'PEDAY']].copy()  # <-- Use .copy() to avoid issues

df_pe.loc[:, 'HEENT_Class'] = df_pe['PEHEENT'].apply(lambda x: 1 if x == 'Abnormal' else 0)
df_pe['PEHEENT'] = df_pe['PEHEENT'].str.strip()
df_pe.loc[:, 'Cardio_Class'] = df_pe['PECardio'].apply(lambda x: 1 if x == 'Abnormal' else 0)
df_pe['PECardio'] = df_pe['PECardio'].str.strip()
df_pe.loc[:, 'Resp_Class'] = df_pe['PEResp'].apply(lambda x: 1 if x == 'Abnormal' else 0)
df_pe['PEResp'] = df_pe['PEResp'].str.strip()
df_pe.loc[:, 'GI_Class'] = df_pe['PEGI'].apply(lambda x: 1 if x == 'Abnormal' else 0)
df_pe['PEGI'] = df_pe['PEGI'].str.strip()
df_pe.loc[:, 'Hepatic_Class'] = df_pe['PEHepatic'].apply(lambda x: 1 if x == 'Abnormal' else 0)
df_pe['PEHepatic'] = df_pe['PEHepatic'].str.strip()
df_pe.loc[:, 'GU_Class'] = df_pe['PEGU'].apply(lambda x: 1 if x == 'Abnormal' else 0)
df_pe['PEGU'] = df_pe['PEGU'].str.strip()
df_pe.loc[:, 'Renal_Class'] = df_pe['PERenal'].apply(lambda x: 1 if x == 'Abnormal' else 0)
df_pe['PERenal'] = df_pe['PERenal'].str.strip()
df_pe.loc[:, 'Neuro_Class'] = df_pe['PENeuro'].apply(lambda x: 1 if x == 'Abnormal' else 0)
df_pe['PENeuro'] = df_pe['PENeuro'].str.strip()
df_pe.loc[:, 'Psych_Class'] = df_pe['PEPsych'].apply(lambda x: 1 if x == 'Abnormal' else 0)
df_pe['PEPsych'] = df_pe['PEPsych'].str.strip()
df_pe.loc[:, 'Endo_Class'] = df_pe['PEEndo'].apply(lambda x: 1 if x == 'Abnormal' else 0)
df_pe['PEEndo'] = df_pe['PEEndo'].str.strip()
df_pe.loc[:, 'Heme_Class'] = df_pe['PEHema'].apply(lambda x: 1 if x == 'Abnormal' else 0)
df_pe['PEHema'] = df_pe['PEHema'].str.strip()
df_pe.loc[:, 'MSK_Class'] = df_pe['PEMusculo'].apply(lambda x: 1 if x == 'Abnormal' else 0)
df_pe['PEMusculo'] = df_pe['PEMusculo'].str.strip()
df_pe.loc[:, 'Derm_Class'] = df_pe['PEDerm'].apply(lambda x: 1 if x == 'Abnormal' else 0)
df_pe['PEDerm'] = df_pe['PEDerm'].str.strip()
df_pe.loc[:, 'Allergy_Class'] = df_pe['PEAllergies'].apply(lambda x: 1 if x == 'Abnormal' else 0)
df_pe['PEAllergies'] = df_pe['PEAllergies'].str.strip()
df_pe.loc[:, 'Immune_Class'] = df_pe['PEImmune'].apply(lambda x: 1 if x == 'Abnormal' else 0)
df_pe['PEImmune'] = df_pe['PEImmune'].str.strip()
df_pe.loc[:, 'Alcohol_Class'] = df_pe['PEAlcohol'].apply(lambda x: 1 if x == 'Abnormal' else 0)
df_pe['PEAlcohol'] = df_pe['PEAlcohol'].str.strip()

#display head 
df_pe.head()

In [ ]:
# Above converted to a function
def one_hot_encode(df, finding_columns):
    """
    One-hot encodes columns from df. If 'Abnormal', then 1.
    Strips whitespace from original column
    Returns df
    """

    for col in finding_columns:
        class_col = f'{col[2:]}_Class'  # Create class column name (e.g., HEENT_Class)
        df.loc[:, class_col] = df[col].apply(lambda x: 1 if x == 'Abnormal' else 0)
        df[col] = df[col].str.strip()

    return df

df_pe = df_pe_original[['PUDID', 'PETime', 'PEHEENT', 'PEHEENTDesc', 'PECardio', 'PECardioDesc', 
                        'PEResp', 'PERespDesc', 'PEGI', 'PEGIDesc', 'PEHepatic', 'PEHepaticDesc', 
                        'PEGU', 'PEGUDesc', 'PERenal', 'PERenalDesc', 'PENeuro', 'PENeuroDesc', 
                        'PEPsych', 'PEPsychDesc', 'PEEndo', 'PEEndoDesc', 'PEHema', 'PEHemaDesc', 
                        'PEMusculo', 'PEMusculoDesc', 'PEDerm', 'PEDermDesc', 'PEAllergies', 
                        'PEAllergiesDesc', 'PEImmune', 'PEImmuneDesc', 'PEAlcohol', 'PEAlcoholDesc', 
                        'PEDAY']].copy()
finding_columns = [
    'PEHEENT', 'PECardio', 'PEResp', 'PEGI', 'PEHepatic', 'PEGU', 'PERenal', 'PENeuro', 'PEPsych', 'PEEndo', 'PEHema', 'PEMusculo', 
    'PEDerm','PEAllergies', 'PEImmune', 'PEAlcohol']

one_hot_encode(df_pe, finding_columns)
df_pe.head()

## Regex Section

In [ ]:
#Data cleaning

#should we have different df for each system? I do not think so but will need to do a lot of replacing 
#HEENT - going to have to combine everything that says a version of dry mouth, dry mucous etc. into a single term - even some things that say 'Tacky M'
#Cardio - tachycardic and all versions of that spelling
#Resp - Kussmaul and all versions of that spelling, tachypnic 
#GI - all terms associated with abdominal tenderness, nausea/emesis
#GU - polyuria and anything mentioning increased urine output
#Renal - polyuria as above, will have to be careful not to double count 
#Neuro - this part is going to be really hard, possibly should just use GCS for this part 
#Psych - Tired, altered, etc. 
#Endo - hyperglycemia/history of DKA?

In [ ]:
def create_binary_flag(df, description_column, new_column, search_patterns, regex=False):
    """
    Creates a binary flag based on the presence of specified search patterns in the description column.
    regex = False -> treated as literal strings. When regex = True, you can use regex in the search_patterns
    """

    if description_column not in df.columns:
        raise ValueError(f"'{description_column}' not found in {df} columns.")
    if not search_patterns:
        return df

    df[new_column] = 0
    #df.loc[df[description_column].isna(), new_column] = np.nan # We don't need to put nulls in

    for pattern in search_patterns:
        df.loc[df[description_column].str.lower() == pattern.lower(), new_column] = 1

    for pattern in search_patterns:
        if regex:
            df.loc[df[description_column].str.contains(pattern, case=False, na=False, regex=True) & (df[new_column] == 0), new_column] = 1
        else:
            df.loc[df[description_column].str.contains(pattern, case=False, na=False, regex=False) & (df[new_column] == 0), new_column] = 1

    return df

In [ ]:
# HEENT Dry_MM
#df_pe = create_binary_flag(df_pe, 'PEHEENTDesc', 'HEENT_DryMM', ["dry muc", "drymm", "dry lip"], regex=True) # original line
ENT_search = r'(?:dry|muc(?:ous|us|ous membrane|us membrane|mm)|mm|tacky)'
df_pe = create_binary_flag(df_pe, 'PEHEENTDesc', 'HEENT_DryMM', [ENT_search], regex=True)

#print(df_pe.loc[df_pe['PEHEENT'] == 'Abnormal', ['PEHEENT', 'PEHEENTDesc', 'HEENT_DryMM']].head(50)) # Test print

In [ ]:
# Cardio Tachy:
#df_pe = create_binary_flag(df_pe, 'PECardioDesc', 'Cardio_Tachy', ["tachycardic", "tachycardia"]) # Original
df_pe = create_binary_flag(df_pe, 'PECardioDesc', 'Cardio_Tachy', [r'(?:tachy(?!p)\w*|elevated)'], regex=True)

#print(df_pe.loc[df_pe['PECardio'] == 'Abnormal', ['PECardio', 'PECardioDesc', 'Cardio_Tachy']].head(50))

In [ ]:
column_name = 'PERespDesc'

all_terms = df_pe[column_name].str.split(expand=True).stack().str.lower()

term_counts = all_terms.value_counts()

print(f"Top 10 Terms in '{column_name}':")
for term, count in term_counts.head(10).items():
    print(f"{term}: {count}")

In [ ]:
# Resp Tachy - we will need to include all cases or do regex to figure out how to do it with capitals etc.
#df_pe = create_binary_flag(df_pe, 'PERespDesc', 'Resp_Tachy', ['tachypnea','Tachypnea','tachycardia', 'Tachypneic'])
df_pe = create_binary_flag(df_pe, 'PERespDesc', 'Resp_Tachy', [r'tachy\w*|kuss\w*'], regex=True)

#print(df_pe.loc[df_pe['PEResp'] == 'Abnormal', ['PEResp', 'PERespDesc', 'Resp_Tachy']].head(50))

In [ ]:
# Resp Kuss: This is going to have to contain every version of the spelling because it is spelled wrong a lot 
df_pe = create_binary_flag(df_pe, 'PERespDesc', 'Resp_Kuss', [r'kuss\w*'], regex=True)

#print(df_pe.loc[df_pe['PEResp'] == 'Abnormal', ['PEResp', 'PERespDesc', 'Resp_Kuss']].head(50))

In [ ]:
# GI - Tender

#GI - Nausea/Vomiting

#GU - Polyuria + Frequency + Urinating alot + Increased Urination

#Renal - we might have to combine renal and GU because they both say polyuria (not sure if you can do that with create_binary

#Neuro - Sleepy/Lethargic/Altered Mental Status/Somnolent/Listless/Fatigued/Tired/Drowsy

#Psych - Agitated/Distressed/Irritable/

#Endo - I am not sure what to do with this one but I am inclined to leave it out or only use ketotic breath/acetone breath

#MSK - Thin - 'Cachectic'- 'Thin-appearing' 'Thin extremities' etc. 

#MSK - Muscle Pain - 'Myalgia' 'Muscle Pain' 

# DERM - Pallor/Pale skin

# DERM - Dry skin


df_pe.head(10)

## Date Time
****Converting Physical Exam Time to Date Time****

In [ ]:
#covnerting PETime to datetime

#first convert to string for date time function
df_pe['PETime'] = (df_pe['PETime']).astype(str)

#some values are single digits - assuming there are leading zeros
df_pe.loc[df_pe["PETime"] == "0", "PETime"] = "0000"
df_pe.loc[df_pe["PETime"] == "1", "PETime"] = "0001"
df_pe.loc[df_pe["PETime"] == "2", "PETime"] = "0002"
df_pe.loc[df_pe["PETime"] == "4", "PETime"] = "0004"
df_pe.loc[df_pe["PETime"] == "9", "PETime"] = "0009"

#convert to time
df_pe['PETime'] = pd.to_datetime(df_pe['PETime'], format='%H%M').dt.time# Print the DataFrameprint(df)
df_pe.head()

**Imaging data**

In [ ]:
#load in imaging data
imaging = pd.read_csv("IMAGING.csv")
imaging.head()

In [ ]:
#change time to datetime
imaging['ImageTime'] = imaging['ImageTime'].astype(str).str.zfill(4) #add leading zeros where time is less than 4 digits
#convert to time
imaging['ImageTime'] = pd.to_datetime(imaging['ImageTime'], format='%H%M').dt.time # Print the DataFrameprint(df)

**Merge physical exam and imaging dataframes**

In [ ]:
#merge dataframes
df_pe_imaging = pd.merge(df_pe, imaging, how ='outer', on ='PUDID') 

#drop extra columns
df_pe_imaging.drop(['StudyEvent','Occurrence', 'ItemGroupRepeatKey'], axis = 1)
df_pe_imaging.head()
#df_pe_imaging.shape #there are some patients with multiple imaging times

***Converting imaging times to actual date times based on IMAGEDAY and PEDAY***

In [ ]:
#creating reference date - change if there is a real reference date for each patient visit

df_pe_imaging["ImageTime"] = df_pe_imaging["ImageTime"].astype(str)
df_pe_imaging["PETime"] = df_pe_imaging["PETime"].astype(str)

# Convert ImageTime to timedelta
df_pe_imaging["ImageTime"] = pd.to_timedelta(df_pe_imaging["ImageTime"])
df_pe_imaging["PETime"] = pd.to_timedelta(df_pe_imaging["PETime"])

# Set reference date
reference_date = pd.to_datetime("2025-01-01")

# Compute full datetime
df_pe_imaging["ImagingDateTime"] = reference_date + pd.to_timedelta(df_pe_imaging["IMAGEDAY"], unit="D") + df_pe_imaging["ImageTime"]
df_pe_imaging["ExamDateTime"] = reference_date + pd.to_timedelta(df_pe_imaging["PEDAY"], unit="D") + df_pe_imaging["PETime"]

df_pe_imaging.head(20)

*Demographics*
* Sex = Sex
* Ethnicity = Ethnicity 
* Race 1 = American Indian or Alaska Native 
* Race 2 = Asian
* Race 3 = Black or African American
* Race 3 = Native Hawaiian or Other Pacific Islander 
* Race 5 = White
* Race 92 = Unknown or Not Reported
* AgeInYears = Age in years

In [ ]:
df_demo = pd.read_csv('Demographics.csv')

df_demo = df_demo[['PUDID', 'Sex', 'Ethnicity', 'Race1', 'Race2', 'Race3', 'Race4', 'Race5', 'Race92', 'AgeInYears']]

#Rename the columns 
df_demo = df_demo.rename(columns={
    'Race1': 'American Indian or Alaska Native',
    'Race2': 'Asian',
    'Race3': 'Black or African American',
    'Race4': 'Native Hawaiian or Other Pacific Islander',
    'Race5': 'White',
    'Race92': 'Unknown or Not Reported',
    'AgeInYears': 'Age'
})

#Total number of patients
total = len(df_demo['PUDID'])
print("Total number of patients:", total)

# Create a dictionary to store value counts for each category
summary_data = {
    'Sex': df_demo['Sex'].value_counts(),
    'Ethnicity': df_demo['Ethnicity'].value_counts(),
    'American Indian or Alaska Native': df_demo['American Indian or Alaska Native'].value_counts(), 
    'Asian': df_demo['Asian'].value_counts(),
    'Black or African American': df_demo['Black or African American'].value_counts(),
    'Native Hawaiian or Other Pacific Islander': df_demo['Native Hawaiian or Other Pacific Islander'].value_counts(),
    'White': df_demo['White'].value_counts(),
    'Unknown or Not Reported': df_demo['Unknown or Not Reported'].value_counts(),
}

#Convert dictionary to DataFrame
summary_df = pd.DataFrame(summary_data)

# Display the summary table
print(summary_df)

df_demo['Age'].describe(percentiles = [.25, .5, .75])

*Creating simple figures for demographics*

In [ ]:
#Changing Race to 1/0s for easier plotting
df_demo[['American Indian or Alaska Native', 'Asian', 'Black or African American', 'Native Hawaiian or Other Pacific Islander',
'White','Unknown or Not Reported']] = (df_demo[['American Indian or Alaska Native', 'Asian', 'Black or African American', 'Native Hawaiian or Other Pacific Islander',
'White','Unknown or Not Reported']]).replace({"Yes":1, "No":0})
df_demo.head()

In [ ]:
#import matplot lib
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator

In [ ]:
#histogram for age
fig = plt.figure(figsize = (10,6))

fig, ax = plt.subplots()
ax.hist(df_demo['Age'],
        bins = 19,
        range = (0,18),
                color = 'steelblue',
       edgecolor = 'white') 
ax.set_title("Age of DKA Patients")
ax.set_ylabel("Patient Count")
ax.text(0.2, 0.9, "Mean age (SD) = 11.6 (4.1)", fontsize=10, ha="center", transform=ax.transAxes)

plt.show()

In [ ]:
#create race counts

#subset dataframe for just race columns
race_counts = df_demo[['American Indian or Alaska Native', 'Asian', 'Black or African American', 'Native Hawaiian or Other Pacific Islander',
'White','Unknown or Not Reported']].sum()

fig, ax = plt.subplots(figsize = (10,6))
ax.bar(race_counts.index, race_counts.values,
                color = 'cornflowerblue')
       
ax.set_title("Patient Race")
ax.set_ylabel("Count")
plt.xticks(rotation = 45, ha = 'right')

plt.show()

In [ ]:
#HEENT

# #Create Dry_MM column
# df['HEENT_DryMM'] = 0

# # #Apply sequential filters
# df.loc[df['PEHEENTDesc'].str.contains('dry muc', case=False, na=False), 'HEENT_DryMM'] = 1
# df.loc[df['PEHEENTDesc'].str.contains('drymm', case=False, na=False), 'HEENT_DryMM'] = 1
# df.loc[df['PEHEENTDesc'].str.contains('dry lip', case=False, na=False), 'HEENT_DryMM'] = 1


# #print(df[df['PEHEENTDesc'].str.contains("dry mucous", case = False, na = False)])
# # df.dtypes
# df.head(20)

#Cardio

#Create tachycardio column
df['Cardio_Tachy'] = 0

#Apply sequential filters
df.loc[df['PECardioDesc'] == 'tachycardic', 'Cardio_Tachy'] = 1
df.loc[df['PECardioDesc'] == 'tachycardia', 'Cardio_Tachy'] = 1
df.loc[df['PECardioDesc'].str.contains('tachycardic', case=False, na=False), 'Cardio_Tachy'] = 1
df.loc[df['PECardioDesc'].str.contains('tachycardia', case=False, na=False), 'Cardio_Tachy'] = 1

# df.dtypes
df.head(20)

##Resp 

#Create tachypnea column
df['Resp_Tachy'] = 0

#Create Kussmaul
df['Resp_Kuss'] = 0

#Apply sequential filters to resp_tachy
df.loc[df['PERespDesc'] == 'tachypnea', 'Resp_Tachy'] = 1
df.loc[df['PERespDesc'] == 'tachycardia', 'Resp_Tachy'] = 1

#Apply sequential filters to resp_kuss
#df.loc[df['PERespDesc'] == 'Kussmaul', 'Resp_Kuss'] = 1
df.loc[df['PERespDesc'].str.contains('Kuss', case=False, na=False), 'Resp_Kuss'] = 1

df.head(10)

##GI 

#Create abdominal pain column
df['GI_Pain'] = 0

#Create Kussmaul
df[''] = 0